In [ ]:

import os
import kagglehub

data_path = kagglehub.dataset_download('sahilchambyal/solana-price-usd')

# List the files inside the downloaded data directory
for dirname, _, filenames in os.walk(data_path):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/root/.cache/kagglehub/datasets/sahilchambyal/solana-price-usd/versions/1/SOLUSD_1s_01NOV2024_to15NOV2024.csv
/root/.cache/kagglehub/datasets/sahilchambyal/solana-price-usd/versions/1/SOLUSD_1s_05NOV2024_to15NOV2024.csv
/root/.cache/kagglehub/datasets/sahilchambyal/solana-price-usd/versions/1/SOLUSD_1min_20AUG2020_to15NOV2024.csv


In [ ]:
import pandas as pd
df = pd.read_csv(data_path + '/SOLUSD_1min_20AUG2020_to15NOV2024.csv')

In [ ]:
# prompt: help me extract a single array of data consisting of only opening prices from the df

opening_prices = df['Open'].values

In [ ]:
def calculate_average_percentage_change(prices, x, y):
    n = len(prices)
    total_percentage_change = 0
    total_pairs = 0
    num_ranges = n // y
    for r in range(num_ranges):
        start = r * y
        end = start + y
        range_prices = prices[start:end]
        m = len(range_prices)
        if m == 0:
            continue

        max_k = x  # number of transactions
        buy = [[0]*m for _ in range(max_k+1)]
        sell = [[0]*m for _ in range(max_k+1)]

        for k in range(max_k+1):
            buy[k][0] = -range_prices[0]
            sell[k][0] = 0

        for i in range(1, m):
            for k in range(1, max_k+1):
                buy[k][i] = max(buy[k][i-1], sell[k-1][i-1] - range_prices[i])
                sell[k][i] = max(sell[k][i-1], buy[k][i-1] + range_prices[i])

        # Reconstruct the transactions
        transactions = []
        k = max_k
        i = m -1
        while k > 0 and i > 0:
            if sell[k][i] == sell[k][i-1]:
                i -=1
            else:
                sell_day = i
                # Find the buy day
                while i > 0 and buy[k][i] == buy[k][i-1]:
                    i -=1
                buy_day = i
                transactions.append((start+buy_day, start+sell_day, range_prices[buy_day], range_prices[sell_day]))
                k -=1
                i -=1

        # Reverse transactions to have them in chronological order
        transactions = transactions[::-1]

        for buy_day, sell_day, buy_price, sell_price in transactions:
            percentage_change = (sell_price - buy_price) / buy_price
            total_percentage_change += percentage_change
            total_pairs +=1

    if total_pairs == 0:
        return 0
    average_percentage_change = total_percentage_change / total_pairs
    return average_percentage_change * 100  # If you want to express it in percentage

# Example usage:
prices = opening_prices
# x = 1 # Number of trades(buy_sell) in a day
x = int(input('Number of Trades in a range : ')) # Number of trades(buy_sell) in a day
# y = 1440 # Range in dataset, currently set to a day (1440 Minutes)
y = int(input(f'Range of minutes to find {x} trades in : ')) # Range in dataset, currently set to a day (1440 Minutes)
# prices = [float(x) for x in input("Enter stock prices separated by spaces: ").split()]
# x = int(input("Enter number of pairs (x): "))
# y = int(input("Enter range size (y): "))

average_change = calculate_average_percentage_change(prices, x, y)
print(f"The average percentage change is: {average_change}%")
print(f'Total gain averagely in every {y} minutes is {x * average_change}%')


Number of Trades in a range : 10
Range of minutes to find 10 trades in : 1440
The average percentage change is: 2.4738914286486846%
Total gain averagely in every 1440 minutes is 24.738914286486846%


In [ ]:
df = df.iloc[-10080:]

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def calculate_buy_sell_signals(df, x, y):
    prices = df['Close'].values
    dates = df['CloseTime'].values  # Use 'OpenTime' if preferred
    n = len(prices)
    signals = []  # List to store buy and sell signals

    num_ranges = n // y
    for r in range(num_ranges):
        start = r * y
        end = start + y
        range_prices = prices[start:end]
        range_dates = dates[start:end]
        m = len(range_prices)
        if m == 0:
            continue

        max_k = x  # number of transactions
        buy = np.full((max_k+1, m), -np.inf)
        sell = np.full((max_k+1, m), -np.inf)

        for k in range(max_k+1):
            buy[k][0] = -range_prices[0]
            sell[k][0] = 0

        for i in range(1, m):
            for k in range(1, max_k+1):
                buy[k][i] = max(buy[k][i-1], sell[k-1][i-1] - range_prices[i])
                sell[k][i] = max(sell[k][i-1], buy[k][i-1] + range_prices[i])

        # Reconstruct the transactions
        transactions = []
        k = max_k
        i = m - 1
        while k > 0 and i >= 0:
            if sell[k][i] == sell[k][i-1]:
                i -= 1
            else:
                sell_day = i
                sell_price = range_prices[sell_day]
                sell_date = range_dates[sell_day]
                # Find the buy day
                while i > 0 and buy[k][i] == buy[k][i-1]:
                    i -= 1
                buy_day = i
                buy_price = range_prices[buy_day]
                buy_date = range_dates[buy_day]
                transactions.append({
                    'buy_day': start + buy_day,
                    'sell_day': start + sell_day,
                    'buy_price': buy_price,
                    'sell_price': sell_price,
                    'buy_date': buy_date,
                    'sell_date': sell_date,
                })
                k -= 1
                i -= 1

        # Reverse transactions to have them in chronological order
        transactions = transactions[::-1]

        signals.extend(transactions)

    return signals

# Example usage:
# Assuming df is your dataframe and you have imported it
# x = number of maximum difference pairs (buy, sell)
# y = range size
x = int(input("Enter number of trades (x): "))
y = int(input("Enter range of minutes (y): "))

signals = calculate_buy_sell_signals(df, x, y)

# Convert dates to datetime if not already
df['CloseTime'] = pd.to_datetime(df['CloseTime'], unit='ms')

# Create the plot
fig = go.Figure()

# Add the price line
fig.add_trace(go.Scatter(
    x=df['CloseTime'],
    y=df['Close'],
    mode='lines',
    name='Close Price'
))

# Prepare buy signals
buy_dates = [pd.to_datetime(s['buy_date'], unit='ms') for s in signals]
buy_prices = [s['buy_price'] for s in signals]

# Add buy signals
fig.add_trace(go.Scatter(
    x=buy_dates,
    y=buy_prices,
    mode='markers',
    marker_symbol='triangle-up',
    marker_color='green',
    marker_size=10,
    name='Buy Signal'
))

# Prepare sell signals
sell_dates = [pd.to_datetime(s['sell_date'], unit='ms') for s in signals]
sell_prices = [s['sell_price'] for s in signals]

# Add sell signals
fig.add_trace(go.Scatter(
    x=sell_dates,
    y=sell_prices,
    mode='markers',
    marker_symbol='triangle-down',
    marker_color='red',
    marker_size=10,
    name='Sell Signal'
))

# Update layout
fig.update_layout(
    title='Stock Price with Buy and Sell Signals',
    xaxis_title='Date',
    yaxis_title='Price',
    legend_title='Legend',
    hovermode='x unified'
)

# Show the plot
fig.show()


Enter number of trades (x): 10
Enter range of minutes (y): 1440
